In [1]:
!nvidia-smi

Tue Feb 11 03:32:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os


In [ ]:
!pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 9.3 MB/s eta 0:00:00


In [ ]:
!pip install .

Processing /content
  Preparing metadata (setup.py) ... done
  Created wheel for custom_kernels: filename=custom_kernels-0.0.1-cp311-cp311-linux_x86_64.whl size=2588545 sha256=ed4e7b2463f26ad5d25b4de03209da2901737b25bb041d09d3b8fea31e30018b
  Stored in directory: /tmp/pip-ephem-wheel-cache-f77tdqlx/wheels/01/d1/e4/ca90c6fac4331f6da6de5353843d0b67505c2bbc8768ac296e
Successfully built custom_kernels
  Attempting uninstall: custom_kernels
    Found existing installation: custom_kernels 0.0.1
    Uninstalling custom_kernels-0.0.1:
      Successfully uninstalled custom_kernels-0.0.1


In [48]:
%%writefile sum_reduction.cu

#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>

__global__ void reduce_global(float *d_in, float *d_out, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int tid = threadIdx.x;

    if (idx >= N) return;


    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s && idx + s < N) {
            d_in[idx] += d_in[idx + s];
        }
        __syncthreads();
    }


    if (tid == 0) {
        d_out[blockIdx.x] = d_in[idx];
    }
}

void reduce_cpu(float *h_in, float *h_out, int N) {
    h_out[0] = 0;
    for (int i = 0; i < N; i++) {
        h_out[0] += h_in[i];
    }
}

int main() {
    const int N = 1 << 22;
    size_t size = N * sizeof(float);

    float *h_in = (float *)malloc(size);
    for (int i = 0; i < N; i++) {
        h_in[i] = 1.0f;  // Initialize for known sum
    }

    size_t num_threads = 1024;
    int num_blocks = (N + num_threads - 1) / num_threads;
    float *h_out = (float *)malloc((num_blocks + 1) * sizeof(float));

    float *d_in, *d_out;
    cudaMalloc((void **)&d_in, size);
    cudaMalloc((void **)&d_out, (num_blocks + 1) * sizeof(float));

    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);


    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);



    reduce_global<<<num_blocks, num_threads>>>(d_in, d_out, N);
    cudaMemcpy(h_out, d_out,num_blocks*sizeof(float), cudaMemcpyDeviceToHost);

    reduce_global<<<1, num_blocks>>>(d_out, d_out, num_blocks);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float gpu_time;
    cudaEventElapsedTime(&gpu_time, start, stop);

    cudaMemcpy(h_out, d_out, sizeof(float), cudaMemcpyDeviceToHost);
    float gpu_sum = h_out[0];


    auto cpu_start = std::chrono::high_resolution_clock::now();
    reduce_cpu(h_in, h_out, N);
    auto cpu_end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> cpu_time = cpu_end - cpu_start;

    float cpu_sum = h_out[0];

    if (gpu_sum == cpu_sum) {
        printf("Success! GPU and CPU sums match: %.2f\n", gpu_sum);
    } else {
        printf("SUM mismatch! GPU: %.2f, CPU: %.2f\n", gpu_sum, cpu_sum);
    }

    printf("GPU Time: %.3f ms\n", gpu_time);
    printf("CPU Time: %.3f ms\n", cpu_time.count());


    cudaFree(d_in);
    cudaFree(d_out);
    free(h_in);
    free(h_out);

    return 0;
}





Overwriting sum_reduction.cu


In [49]:
!nvcc sum_reduction.cu -o abc -arch=sm_75

In [50]:
!./abc

SUM mismatch! GPU: 1024.00, CPU: 4194304.00
GPU Time: 0.617 ms
CPU Time: 12.262 ms
